In [1]:
import wget
import pandas as pd

In [2]:
spacex_launch_dash = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_dash_app = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")

In [33]:
# %load spacex_dash_app.py
# Import required libraries
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

spacex_df = pd.read_csv("spacex_launch_dash.csv")
LS = spacex_df['Launch Site'].unique()

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options = [{'label':'All Sites','value':'All'}, 
                                                       {'label':LS[0] , 'value':LS[0]},
                                                       {'label':LS[1] , 'value':LS[1]},
                                                       {'label':LS[2] , 'value':LS[2]},
                                                       {'label':LS[3] , 'value':LS[3]},
                                                      ],
                                                       value='ALL',
                                                       placeholder="place holder here",
                                                       searchable=True
                                                        ), 
                        
                                html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                            
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       100: '100'},
                                                value=[min_payload, max_payload]),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filter_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filter_df, 
                    values='class', 
                    names='Launch Site', 
                    title= 'Total Success Launches for Site',
                    )
        return fig
    else:
        # return the outcomes pichart for a selected site
        filter_df = spacex_df[spacex_df['Launch Site']==entered_site]
        filter_df = filter_df.groupby(['Launch Site', 'class']).size().reset_index()
        filter_df.rename(columns={0:'class count'}, inplace=True)
        fig = px.pie(filter_df,
            values='class count', 
            names= 'class',
            title = f'Total Success Launches for Site {entered_site}'
                    )
        return fig 
    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
#Two Input components: one to receive selected launch site and another to receive selected payload range
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")] 
             )

def build_scatter(site, payload):
    df = spacex_df 
    if  site == 'ALL':
        low, high = payload
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        
        fig = px.scatter(df[mask], x="Payload Mass (kg)", y="class", color="Booster Version Category")
    else:
        low, high = payload
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        
        df = df.loc[df['Launch Site'] == site, ['Payload Mass (kg)', 'class', 'Booster Version Category']]
        fig = px.scatter(df[mask], x="Payload Mass (kg)", y="class", color="Booster Version Category")
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Aug/2022 10:47:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 10:47:57] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 10:47:57] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 10:47:57] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 10:47:57] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 10:47:57] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 10:47:57] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 10:47:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 10:47:57] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\Safa\AppData\Local\Temp\ipykernel_9380\393600914.py:103: UserWarning:

Boolean Series key will be reindexed to match DataFra